In [1]:
%matplotlib inline

In [18]:
from torch.utils.data import Dataset
from os.path import join
from os import listdir 
from PIL import Image
import numpy as np 
import torch        
import numpy as np 
import torch 
from sumnist import SuMNIST
from pytorch_ood.utils import fix_random_seed

fix_random_seed(123)
device = "cuda:0"
all_results = []


In [3]:
import matplotlib.pyplot as plt 
from utils.visualize import draw_boxes

In [4]:
from torchvision.transforms.functional import to_pil_image

data = SuMNIST(root="../sumnist", train=True)

img, target = data[0]

draw_boxes(
    np.array(to_pil_image(img)), 
    scores=torch.ones(target["labels"].shape),
    boxes=target["boxes"].detach().cpu(),
    labels=target["labels"].detach().cpu(),
    captions = [str(x.item()) for x in target["labels"].detach().cpu()]
    )

plt.show()

In [5]:
import torchvision
from torchvision.models.detection.faster_rcnn import FastRCNNPredictor
from torchvision.models.detection import FasterRCNN
from torchvision.models.detection.rpn import AnchorGenerator

def my_forward(self, x: torch.Tensor) -> torch.Tensor:
    x = self.conv1(x)
    x = self.bn1(x)
    x = self.relu(x)
    x = self.maxpool(x)

    x = self.layer1(x)
    x = self.layer2(x)
    x = self.layer3(x)
    x = self.layer4(x)
    return x 
    
def get_model_object_detection(num_classes):
    backbone = torchvision.models.resnet18(pretrained=True)

    # monkeypatching the forward method to return features 
    backbone.forward = my_forward.__get__(backbone, type(backbone))
    
    # output channels in a backbone. For mobilenet_v2, it's 1280
    # backbone.out_channels = 1280
    backbone.out_channels = 512

    # let's make the RPN generate 5 x 3 anchors per spatial
    # location, with 5 different sizes and 3 different aspect
    # ratios. We have a Tuple[Tuple[int]] because each feature
    # map could potentially have different sizes and
    # aspect ratios
    anchor_generator = AnchorGenerator(sizes=((32, 64, 128, 256, 512),),
                                       aspect_ratios=((0.5, 1.0, 2.0),))

    # let's define what are the feature maps that we will
    # use to perform the region of interest cropping, as well as
    # the size of the crop after rescaling.
    # if your backbone returns a Tensor, featmap_names is expected to
    # be [0]. More generally, the backbone should return an
    # OrderedDict[Tensor], and in featmap_names you can choose which
    # feature maps to use.
    roi_pooler = torchvision.ops.MultiScaleRoIAlign(featmap_names=['0'],
                                                    output_size=7,
                                                    sampling_ratio=2)

    # put the pieces together inside a FasterRCNN model
    model = FasterRCNN(backbone,
                       num_classes=num_classes,
                       rpn_anchor_generator=anchor_generator,
                       box_roi_pool=roi_pooler)
    return model 

In [6]:
import utils.transforms as T

def get_transform(train):
    transforms = []
    # transforms.append(T.PILToTensor())
    transforms.append(T.ConvertImageDtype(torch.float))
    
    if train:
        transforms.append(T.ScaleJitter(target_size=(56,56)))
        # transforms.append(T.RandomHorizontalFlip(0.5))
    
    return T.Compose(transforms)

In [7]:
from utils.engine import train_one_epoch, evaluate
from utils.utils import collate_fn
import traceback

# use our dataset and defined transformations
dataset = SuMNIST(root="../sumnist", train=True, transforms=get_transform(train=True))
dataset_test = SuMNIST(root="../sumnist", train=False, transforms=get_transform(train=False))
    
model_path = "../data/weights/model.pt"

def main():
    # train on the GPU or on the CPU, if a GPU is not available
    device = torch.device(device) if torch.cuda.is_available() else torch.device('cpu')

    num_classes = 10

    # define training and validation data loaders
    data_loader = torch.utils.data.DataLoader(
        dataset, batch_size=16, shuffle=True, num_workers=4,
        collate_fn=collate_fn)

    data_loader_test = torch.utils.data.DataLoader(
        dataset_test, batch_size=8, shuffle=False, num_workers=4,
        collate_fn=collate_fn)

    # get the model using our helper function
    # model = get_model_instance_segmentation(num_classes)
    model = get_model_object_detection(num_classes)
    
    # move model to the right device
    model.to(device)
    
    # construct an optimizer
    params = [p for p in model.parameters() if p.requires_grad]
    optimizer = torch.optim.SGD(params, lr=0.005, momentum=0.9, weight_decay=0.0005)
    # and a learning rate scheduler
    lr_scheduler = torch.optim.lr_scheduler.StepLR(optimizer, step_size=3, gamma=0.1)

    num_epochs = 5

    for epoch in range(num_epochs):
        # train for one epoch, printing every 10 iterations
        train_one_epoch(model, optimizer, data_loader, device, epoch, print_freq=10)
        # update the learning rate
        lr_scheduler.step()
        # evaluate on the test dataset
        try:
            evaluate(model, data_loader_test, device=device)
            torch.save(model, model_path)
        except Exception as e:
            print(e)
            # traceback.print_exception(e)

    print("That's it!")
    return model 


In [8]:
model = main()

/home/kirchhei/.local/lib/python3.10/site-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/home/kirchhei/.local/lib/python3.10/site-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet18_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet18_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


Epoch: [0]  [   0/3750]  eta: 1:49:40  lr: 0.000010  loss: 3.1298 (3.1298)  loss_classifier: 2.2367 (2.2367)  loss_box_reg: 0.1877 (0.1877)  loss_objectness: 0.6784 (0.6784)  loss_rpn_box_reg: 0.0269 (0.0269)  time: 1.7549  data: 0.3580  max mem: 6236
Epoch: [0]  [  10/3750]  eta: 0:34:36  lr: 0.000060  loss: 3.0455 (2.9966)  loss_classifier: 2.1656 (2.1126)  loss_box_reg: 0.1798 (0.1805)  loss_objectness: 0.6778 (0.6774)  loss_rpn_box_reg: 0.0246 (0.0260)  time: 0.5551  data: 0.0489  max mem: 6539
Epoch: [0]  [  20/3750]  eta: 0:30:50  lr: 0.000110  loss: 2.6129 (2.5631)  loss_classifier: 1.7316 (1.6786)  loss_box_reg: 0.1819 (0.1863)  loss_objectness: 0.6736 (0.6729)  loss_rpn_box_reg: 0.0242 (0.0254)  time: 0.4331  data: 0.0161  max mem: 6540
Epoch: [0]  [  30/3750]  eta: 0:29:26  lr: 0.000160  loss: 1.5156 (2.1970)  loss_classifier: 0.6344 (1.3147)  loss_box_reg: 0.1919 (0.1942)  loss_objectness: 0.6571 (0.6639)  loss_rpn_box_reg: 0.0225 (0.0242)  time: 0.4309  data: 0.0142  max me

Epoch: [0]  [ 330/3750]  eta: 0:24:41  lr: 0.001658  loss: 0.4286 (0.9301)  loss_classifier: 0.1984 (0.4445)  loss_box_reg: 0.1950 (0.2613)  loss_objectness: 0.0274 (0.2094)  loss_rpn_box_reg: 0.0101 (0.0149)  time: 0.4324  data: 0.0142  max mem: 6540
Epoch: [0]  [ 340/3750]  eta: 0:24:37  lr: 0.001708  loss: 0.4199 (0.9147)  loss_classifier: 0.1885 (0.4369)  loss_box_reg: 0.1881 (0.2591)  loss_objectness: 0.0245 (0.2040)  loss_rpn_box_reg: 0.0099 (0.0148)  time: 0.4326  data: 0.0143  max mem: 6540
Epoch: [0]  [ 350/3750]  eta: 0:24:33  lr: 0.001758  loss: 0.3965 (0.8998)  loss_classifier: 0.1741 (0.4293)  loss_box_reg: 0.1814 (0.2570)  loss_objectness: 0.0244 (0.1989)  loss_rpn_box_reg: 0.0099 (0.0146)  time: 0.4327  data: 0.0143  max mem: 6540
Epoch: [0]  [ 360/3750]  eta: 0:24:28  lr: 0.001808  loss: 0.3862 (0.8857)  loss_classifier: 0.1737 (0.4223)  loss_box_reg: 0.1814 (0.2550)  loss_objectness: 0.0221 (0.1940)  loss_rpn_box_reg: 0.0095 (0.0145)  time: 0.4322  data: 0.0141  max me

Epoch: [0]  [ 660/3750]  eta: 0:22:19  lr: 0.003307  loss: 0.2525 (0.6261)  loss_classifier: 0.1095 (0.2938)  loss_box_reg: 0.1298 (0.2089)  loss_objectness: 0.0065 (0.1118)  loss_rpn_box_reg: 0.0060 (0.0116)  time: 0.4343  data: 0.0140  max mem: 6540
Epoch: [0]  [ 670/3750]  eta: 0:22:15  lr: 0.003357  loss: 0.2525 (0.6207)  loss_classifier: 0.1130 (0.2912)  loss_box_reg: 0.1308 (0.2078)  loss_objectness: 0.0062 (0.1103)  loss_rpn_box_reg: 0.0060 (0.0115)  time: 0.4349  data: 0.0141  max mem: 6540
Epoch: [0]  [ 680/3750]  eta: 0:22:11  lr: 0.003407  loss: 0.2491 (0.6150)  loss_classifier: 0.1086 (0.2884)  loss_box_reg: 0.1253 (0.2064)  loss_objectness: 0.0056 (0.1087)  loss_rpn_box_reg: 0.0067 (0.0114)  time: 0.4354  data: 0.0142  max mem: 6540
Epoch: [0]  [ 690/3750]  eta: 0:22:07  lr: 0.003457  loss: 0.2233 (0.6095)  loss_classifier: 0.1001 (0.2858)  loss_box_reg: 0.1123 (0.2051)  loss_objectness: 0.0053 (0.1073)  loss_rpn_box_reg: 0.0066 (0.0113)  time: 0.4357  data: 0.0142  max me

Epoch: [0]  [ 990/3750]  eta: 0:19:58  lr: 0.004955  loss: 0.2082 (0.4938)  loss_classifier: 0.0937 (0.2298)  loss_box_reg: 0.1062 (0.1780)  loss_objectness: 0.0029 (0.0763)  loss_rpn_box_reg: 0.0039 (0.0096)  time: 0.4350  data: 0.0139  max mem: 6540
Epoch: [0]  [1000/3750]  eta: 0:19:53  lr: 0.005000  loss: 0.2025 (0.4909)  loss_classifier: 0.0940 (0.2284)  loss_box_reg: 0.1041 (0.1773)  loss_objectness: 0.0033 (0.0756)  loss_rpn_box_reg: 0.0037 (0.0096)  time: 0.4350  data: 0.0140  max mem: 6540
Epoch: [0]  [1010/3750]  eta: 0:19:49  lr: 0.005000  loss: 0.2022 (0.4880)  loss_classifier: 0.0902 (0.2270)  loss_box_reg: 0.1016 (0.1765)  loss_objectness: 0.0028 (0.0749)  loss_rpn_box_reg: 0.0037 (0.0095)  time: 0.4350  data: 0.0140  max mem: 6540
Epoch: [0]  [1020/3750]  eta: 0:19:45  lr: 0.005000  loss: 0.1919 (0.4852)  loss_classifier: 0.0847 (0.2257)  loss_box_reg: 0.1008 (0.1758)  loss_objectness: 0.0035 (0.0742)  loss_rpn_box_reg: 0.0037 (0.0095)  time: 0.4350  data: 0.0140  max me

...

Epoch: [4]  [2270/3750]  eta: 0:10:50  lr: 0.000500  loss: 0.0560 (0.0620)  loss_classifier: 0.0296 (0.0343)  loss_box_reg: 0.0257 (0.0266)  loss_objectness: 0.0002 (0.0006)  loss_rpn_box_reg: 0.0006 (0.0006)  time: 0.4380  data: 0.0140  max mem: 6540
Epoch: [4]  [2280/3750]  eta: 0:10:46  lr: 0.000500  loss: 0.0540 (0.0620)  loss_classifier: 0.0286 (0.0342)  loss_box_reg: 0.0225 (0.0265)  loss_objectness: 0.0002 (0.0006)  loss_rpn_box_reg: 0.0005 (0.0006)  time: 0.4381  data: 0.0140  max mem: 6540
Epoch: [4]  [2290/3750]  eta: 0:10:42  lr: 0.000500  loss: 0.0540 (0.0620)  loss_classifier: 0.0278 (0.0342)  loss_box_reg: 0.0234 (0.0265)  loss_objectness: 0.0002 (0.0006)  loss_rpn_box_reg: 0.0006 (0.0006)  time: 0.4420  data: 0.0159  max mem: 6540
Epoch: [4]  [2300/3750]  eta: 0:10:37  lr: 0.000500  loss: 0.0520 (0.0620)  loss_classifier: 0.0250 (0.0342)  loss_box_reg: 0.0234 (0.0265)  loss_objectness: 0.0003 (0.0006)  loss_rpn_box_reg: 0.0006 (0.0006)  time: 0.4458  data: 0.0179  max me

Epoch: [4]  [2600/3750]  eta: 0:08:25  lr: 0.000500  loss: 0.0524 (0.0620)  loss_classifier: 0.0271 (0.0342)  loss_box_reg: 0.0242 (0.0265)  loss_objectness: 0.0002 (0.0006)  loss_rpn_box_reg: 0.0005 (0.0006)  time: 0.4382  data: 0.0140  max mem: 6540
Epoch: [4]  [2610/3750]  eta: 0:08:21  lr: 0.000500  loss: 0.0507 (0.0621)  loss_classifier: 0.0268 (0.0343)  loss_box_reg: 0.0243 (0.0266)  loss_objectness: 0.0003 (0.0006)  loss_rpn_box_reg: 0.0006 (0.0006)  time: 0.4381  data: 0.0139  max mem: 6540
Epoch: [4]  [2620/3750]  eta: 0:08:17  lr: 0.000500  loss: 0.0666 (0.0621)  loss_classifier: 0.0345 (0.0343)  loss_box_reg: 0.0244 (0.0266)  loss_objectness: 0.0003 (0.0006)  loss_rpn_box_reg: 0.0007 (0.0007)  time: 0.4382  data: 0.0140  max mem: 6540
Epoch: [4]  [2630/3750]  eta: 0:08:12  lr: 0.000500  loss: 0.0633 (0.0621)  loss_classifier: 0.0340 (0.0343)  loss_box_reg: 0.0225 (0.0266)  loss_objectness: 0.0002 (0.0006)  loss_rpn_box_reg: 0.0007 (0.0007)  time: 0.4382  data: 0.0140  max me

Epoch: [4]  [2930/3750]  eta: 0:06:00  lr: 0.000500  loss: 0.0628 (0.0622)  loss_classifier: 0.0352 (0.0344)  loss_box_reg: 0.0264 (0.0266)  loss_objectness: 0.0004 (0.0006)  loss_rpn_box_reg: 0.0007 (0.0006)  time: 0.4385  data: 0.0142  max mem: 6540
Epoch: [4]  [2940/3750]  eta: 0:05:56  lr: 0.000500  loss: 0.0523 (0.0622)  loss_classifier: 0.0315 (0.0344)  loss_box_reg: 0.0214 (0.0266)  loss_objectness: 0.0004 (0.0006)  loss_rpn_box_reg: 0.0005 (0.0006)  time: 0.4384  data: 0.0143  max mem: 6540
Epoch: [4]  [2950/3750]  eta: 0:05:51  lr: 0.000500  loss: 0.0560 (0.0622)  loss_classifier: 0.0303 (0.0344)  loss_box_reg: 0.0247 (0.0266)  loss_objectness: 0.0003 (0.0006)  loss_rpn_box_reg: 0.0007 (0.0006)  time: 0.4385  data: 0.0142  max mem: 6540
Epoch: [4]  [2960/3750]  eta: 0:05:47  lr: 0.000500  loss: 0.0560 (0.0621)  loss_classifier: 0.0311 (0.0343)  loss_box_reg: 0.0246 (0.0266)  loss_objectness: 0.0002 (0.0006)  loss_rpn_box_reg: 0.0005 (0.0006)  time: 0.4384  data: 0.0140  max me

Epoch: [4]  [3260/3750]  eta: 0:03:35  lr: 0.000500  loss: 0.0558 (0.0623)  loss_classifier: 0.0303 (0.0344)  loss_box_reg: 0.0249 (0.0266)  loss_objectness: 0.0003 (0.0006)  loss_rpn_box_reg: 0.0005 (0.0006)  time: 0.4380  data: 0.0140  max mem: 6540
Epoch: [4]  [3270/3750]  eta: 0:03:31  lr: 0.000500  loss: 0.0562 (0.0623)  loss_classifier: 0.0316 (0.0344)  loss_box_reg: 0.0257 (0.0266)  loss_objectness: 0.0002 (0.0006)  loss_rpn_box_reg: 0.0006 (0.0006)  time: 0.4383  data: 0.0141  max mem: 6540
Epoch: [4]  [3280/3750]  eta: 0:03:26  lr: 0.000500  loss: 0.0456 (0.0622)  loss_classifier: 0.0236 (0.0344)  loss_box_reg: 0.0217 (0.0266)  loss_objectness: 0.0002 (0.0006)  loss_rpn_box_reg: 0.0005 (0.0006)  time: 0.4384  data: 0.0142  max mem: 6540
Epoch: [4]  [3290/3750]  eta: 0:03:22  lr: 0.000500  loss: 0.0450 (0.0622)  loss_classifier: 0.0229 (0.0344)  loss_box_reg: 0.0220 (0.0266)  loss_objectness: 0.0002 (0.0006)  loss_rpn_box_reg: 0.0004 (0.0006)  time: 0.4383  data: 0.0141  max me

Epoch: [4]  [3590/3750]  eta: 0:01:10  lr: 0.000500  loss: 0.0498 (0.0622)  loss_classifier: 0.0259 (0.0344)  loss_box_reg: 0.0232 (0.0266)  loss_objectness: 0.0002 (0.0006)  loss_rpn_box_reg: 0.0005 (0.0006)  time: 0.4380  data: 0.0140  max mem: 6540
Epoch: [4]  [3600/3750]  eta: 0:01:05  lr: 0.000500  loss: 0.0498 (0.0623)  loss_classifier: 0.0282 (0.0344)  loss_box_reg: 0.0232 (0.0266)  loss_objectness: 0.0002 (0.0006)  loss_rpn_box_reg: 0.0005 (0.0006)  time: 0.4380  data: 0.0140  max mem: 6540
Epoch: [4]  [3610/3750]  eta: 0:01:01  lr: 0.000500  loss: 0.0723 (0.0623)  loss_classifier: 0.0383 (0.0344)  loss_box_reg: 0.0271 (0.0266)  loss_objectness: 0.0003 (0.0006)  loss_rpn_box_reg: 0.0008 (0.0006)  time: 0.4381  data: 0.0140  max mem: 6540
Epoch: [4]  [3620/3750]  eta: 0:00:57  lr: 0.000500  loss: 0.0723 (0.0623)  loss_classifier: 0.0383 (0.0345)  loss_box_reg: 0.0295 (0.0266)  loss_objectness: 0.0003 (0.0006)  loss_rpn_box_reg: 0.0008 (0.0006)  time: 0.4385  data: 0.0141  max me

In [9]:
model.cuda()

with torch.no_grad():
    model.eval()

    # For Training
    for n, (images, targets) in enumerate(dataset_test):
        print(images.shape)
        if n == 0:
            predictions = model(images.unsqueeze(0).cuda())
            break 

    images = list(image for image in images.unsqueeze(0))

torch.Size([3, 56, 56])


In [10]:
from torchvision.transforms.functional import to_pil_image

print(predictions[0]["scores"].detach().cpu())

draw_boxes(
    np.array(to_pil_image(images[0])), 
    scores=predictions[0]["scores"].detach().cpu(), 
    boxes=predictions[0]["boxes"].detach().cpu(),
    labels=predictions[0]["labels"].detach().cpu(),
    )

tensor([0.9999, 0.9998, 0.9997, 0.9975])


<Axes: >

# Evaluate Model 

In [11]:
model.eval()

FasterRCNN(
  (transform): GeneralizedRCNNTransform(
      Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
      Resize(min_size=(800,), max_size=1333, mode='bilinear')
  )
  (backbone): ResNet(
    (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
    (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (relu): ReLU(inplace=True)
    (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
    (layer1): Sequential(
      (0): BasicBlock(
        (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
        (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (relu): ReLU(inplace=True)
        (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
        (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      )
      (1): Bas

# Hybrid Sum 

In [19]:
import detectors 
from pytorch_ood.utils import OODMetrics

# from importlib import reload
# reload(detectors)

d = detectors.HybridSum(model, value=20)

dataset_test = SuMNIST(root="../sumnist", train=False, transforms=get_transform(train=False))
data_loader = torch.utils.data.DataLoader(dataset_test, batch_size=5, shuffle=False, num_workers=4, collate_fn=collate_fn)

m = OODMetrics()

with torch.no_grad():
    for x, y in data_loader:
        x = [t.to(device) for t in x]
        o = d(x)
        y = torch.where(torch.stack([t["labels"] for t in y]).sum(dim=1) == 20, 0, -1)
        m.update(o, y)


m = m.compute()
m.update({"Method": "HybridSum"})
all_results.append(m)
print(m)


{'AUROC': 0.9841176271438599, 'AUPR-IN': 0.9268642663955688, 'AUPR-OUT': 0.9975858926773071, 'FPR95TPR': 0.029764706268906593, 'Method': 'HybridSum'}


# Hybrid Memory 

In [20]:
import detectors 
# from importlib import reload
# reload(detectors)


dataset = SuMNIST(root="../sumnist", train=True, transforms=get_transform(train=False))


d = detectors.HybridMemory(model)
d.fit(dataset)

dataset_test = SuMNIST(root="../sumnist", train=False, transforms=get_transform(train=False))
data_loader = torch.utils.data.DataLoader(dataset_test, batch_size=5, shuffle=False, num_workers=4, collate_fn=collate_fn)

m = OODMetrics()

with torch.no_grad():
    for x, y in data_loader:
        x = [t.to(device) for t in x]
        o = d(x)
        y = torch.where(torch.stack([t["labels"] for t in y]).sum(dim=1) == 20, 0, -1)
        m.update(o, y)


m = m.compute()
m.update({"Method": "HybridMem"})
all_results.append(m)
print(m)


{'AUROC': 0.9530392289161682, 'AUPR-IN': 0.8272231817245483, 'AUPR-OUT': 0.992926299571991, 'FPR95TPR': 0.09258823841810226, 'Method': 'HybridMem'}


In [21]:
# import matplotlib.pyplot as plt 
# from PIL import Image 
# imgs = [f"output/0000{i}.jpg" for i in range(5)]
# imgs = [Image.open(p) for p in imgs]

In [ ]:
dataset_test = SuMNIST(root="../sumnist", train=False, transforms=get_transform(train=False))
data_loader = torch.utils.data.DataLoader(dataset_test, batch_size=5, shuffle=False, num_workers=4, collate_fn=collate_fn)

with torch.no_grad():
    # For Training
    images, targets = next(iter(data_loader))
    images = torch.stack(images)
    predictions = model(images.cuda())
    
    fig, axs = plt.subplots(1,5, figsize=(10,25))

    for i, p, ax in zip(images, predictions, axs.flatten()):
        draw_boxes(
            i.permute(1,2,0), # np.array(to_pil_image(images[0])), 
            scores=p["scores"].detach().cpu(), 
            boxes=p["boxes"].detach().cpu(),
            # masks=predictions[0]["masks"].detach().cpu(),
            labels=p["labels"].detach().cpu(),
            captions=[str(i.item()) for i in p["labels"]],
            ax=ax
            )
        
        # ax.imshow(imgs[i])
        ax.axis('off')

    plt.tight_layout()
    # plt.axis('off')
    # plt.show()
    plt.savefig("../img/predictions.png", bbox_inches="tight", dpi=300, transparent=True)


In [27]:
import pandas as pd 


df = pd.DataFrame(all_results)
df[["AUROC","AUPR-IN","AUPR-OUT","FPR95TPR",]] *= 100 

print(df.groupby(by="Method").mean().sort_values("AUROC").to_csv(float_format="%.2f"))

Method,AUROC,AUPR-IN,AUPR-OUT,FPR95TPR
HybridMem,95.30,82.72,99.29,9.26
HybridSum,98.41,92.69,99.76,2.98



In [23]:
all_results

[{'AUROC': 0.9841176271438599,
  'AUPR-IN': 0.9268642663955688,
  'AUPR-OUT': 0.9975858926773071,
  'FPR95TPR': 0.029764706268906593,
  'Method': 'HybridSum'},
 {'AUROC': 0.9530392289161682,
  'AUPR-IN': 0.8272231817245483,
  'AUPR-OUT': 0.992926299571991,
  'FPR95TPR': 0.09258823841810226,
  'Method': 'HybridMem'}]